<img src = "../../Data/bgsedsc_0.jpg">

# Project: Decision Trees
### Ignacio Vidal-Quadras Costa


## Import pre-processed data

In [113]:
import pandas as pd
import os

seed = 54
# Training dataset
data=pd.read_csv('pre_processed_data.csv')
data = data.drop('Unnamed: 0', axis =1)
data.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,FIRST_CAREUNIT,LOS,age,weekday,WEEKDAY,SEASON,HOLIDAY,hour,NIGHT,ETHNICS
0,0,55440,195768,228357,89.0,145.0,121.043478,74.0,127.0,106.586957,...,MICU,4.5761,69.608219,6,Sunday,Summer,Non Holiday,10,0,OTHER
1,0,76908,126136,221004,63.0,110.0,79.117647,89.0,121.0,106.733333,...,MICU,0.7582,42.101370,0,Monday,Winter,Non Holiday,22,1,OTHER
2,0,95798,136645,296315,81.0,98.0,91.689655,88.0,138.0,112.785714,...,MICU,3.7626,68.210959,2,Wednesday,Autumn,Non Holiday,23,1,OTHER
3,0,40708,102505,245557,76.0,128.0,98.857143,84.0,135.0,106.972973,...,SICU,3.8734,74.961644,0,Monday,Autumn,Non Holiday,8,0,OTHER
4,0,28424,127337,225281,NaN,NaN,NaN,NaN,NaN,NaN,...,TSICU,5.8654,79.424658,4,Friday,Autumn,Non Holiday,2,1,OTHER


## Import pre-processed test data

In [114]:
# Test dataset (to produce predictions)
data_test=pd.read_csv('pre_processed_data_test.csv')
data_test = data_test.drop('Unnamed: 0', axis =1)
data_test.sort_values('icustay_id').head()

,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,DiasBP_Min,...,ICD9_diagnosis,FIRST_CAREUNIT,age,weekday,WEEKDAY,SEASON,HOLIDAY,hour,NIGHT,ETHNICS
4930,93535,121562,200011,56.0,82.0,71.205128,123.0,185.0,156.411765,37.0,...,49322,MICU,84.287671,3,Thursday,Winter,Non Holiday,20,1,OTHER
1052,30375,177945,200044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85220,SICU,81.964384,6,Sunday,Winter,Non Holiday,16,0,OTHER
3412,73241,149216,200049,54.0,76.0,64.833333,95.0,167.0,114.545455,33.0,...,5722,MICU,64.441096,5,Saturday,Winter,Non Holiday,22,1,OTHER
1725,99052,129142,200063,85.0,102.0,92.560976,91.0,131.0,108.365854,42.0,...,44101,CSRU,37.095890,1,Tuesday,Summer,Non Holiday,23,1,OTHER
981,51698,190004,200081,82.0,133.0,94.323529,86.0,143.0,111.093750,47.0,...,41519,CCU,69.920548,3,Thursday,Summer,Non Holiday,6,1,OTHER


## Import the necessary packages

In [115]:
%matplotlib inline
import matplotlib.pylab as plt

import pandas as pd
import numpy as np
import sklearn
import random
import time
import scipy
import datetime
import imblearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection  import ParameterGrid
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from mlens.ensemble import SuperLearner, BlendEnsemble, Subsemble, SequentialEnsemble

from sklearn.metrics import accuracy_score, mean_squared_error



np.random.seed(3123) # impose random seed for reproducibility

## Classifiaction of our variables and categorical transformation

### Classifiaction of variables
I will eliminate DOD, DISCHTIME because when the patient enters the hospital you cannot know this information (he is still alive)

Subject_id, hadm_id, icustay_id are eliminated because they have no explanatory power

DIAGNOSIS will be droped as I will use the ICD9_diagnosis (because it has fewer observations)

LOS is not included because you do not know it when you get into the hospital

In [116]:
y = data['LOS']

numericals = ['HeartRate_Min','HeartRate_Max','HeartRate_Mean','SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean', 'age', 'com_prob']
categoricals = ['FIRST_CAREUNIT', 'DIAG', 'ADMISSION_TYPE']
#categoricals = ['ADMISSION_TYPE', 'FIRST_CAREUNIT', 'GENDER', 'ETHNICS',
                #'RELIGION', 'INSURANCE','NIGHT', 'WEEKDAY', 'SEASON', 'DIAG', 'HOLIDAY']
#Notice DIAG and com_prob do not exist yet, they will be added in the following steps

variables = numericals + categoricals

### Categorical encoding
I will encode the categoricals using target encoding. I have tried creating dummies but, besides being much slower, the algorythm works worse.

Notice that on this code we don't use the same procedure for ICD9_diagnosis. This is because it might be the case that the diagnosis in the test sample is a new one (does not exist in the train set), in this case I will assign to this variable the general progability of dying.

In [117]:
for n in [i for i in categoricals if i != 'DIAG']:
    dict = data.groupby(n)['HOSPITAL_EXPIRE_FLAG'].mean().to_dict()  
    data[n] = data[n].apply(lambda x: dict[x])
    data_test[n] = data_test[n].apply(lambda x: dict[x])

In [118]:
diag_mean_dict = data.groupby('ICD9_diagnosis')['HOSPITAL_EXPIRE_FLAG'].mean().to_dict()

def get_diag(x):
    if x in diag_mean_dict:
        return diag_mean_dict[x]
    else: 
        return data.HOSPITAL_EXPIRE_FLAG.mean()

data['DIAG'] = data['ICD9_diagnosis'].apply(get_diag)
data_test['DIAG'] = data_test['ICD9_diagnosis'].apply(get_diag)


### Commulative probabilty per patient based on diagnosis
A patient can have come before to the hospital so let's had his probability of dying according to the diagnosis

In [119]:
data['com_prob'] = data.sort_values(['subject_id', 'ADMITTIME']).groupby('subject_id')['DIAG'].cumsum()

data_test['com_prob']=data_test.sort_values(['subject_id', 'ADMITTIME']).groupby('subject_id')['DIAG'].cumsum()

## Select from the train and test only the variables we want

In [120]:
X_train = data[variables]
X_test = data_test[variables]

### Now let's make sure that both datasets have the same columns so that we can apply KNN

In [121]:
X_train.columns.isin(X_test.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [122]:
X_train.columns

Index(['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean',
       'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max',
       'Glucose_Mean', 'age', 'com_prob', 'FIRST_CAREUNIT', 'DIAG',
       'ADMISSION_TYPE'],
      dtype='object')

In [123]:
col_to_use = X_train.columns[X_train.columns.isin(X_test.columns)]
len(col_to_use)

29

In [124]:
col_to_use = X_train.columns[X_train.columns.isin(X_test.columns)]
len(col_to_use)
X_train = X_train[col_to_use]
X_test = X_test[col_to_use]
print(X_test.shape)
print(X_train.shape)

(5221, 29)
(20885, 29)


## Dealing with the missing values

Previous in this assigment I tried different ways of handeling missing data.

I tried deleating all the rows with at least one missing value in the train data set and substituting the missing values in the test with the median of the varibale but my accuracy went down a lot.

Then I also tried the with the KNN method for missing data.

I also tried to subsitute them by just the mean.

Finally, the median turned out to be the most efficient one.


In [125]:
X_train.fillna(X_train.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)

## Feature engenieering
##### (Find the code in the support_file)

I will not explore the skewness and kurtosis of the numerical varibales because the models I am implementing are ¡ and XGBoost and random forest, since decision trees do not require normalization of their inputs and both random forests and XGBoost are a variation of decision trees, they do not require normalization of their imnputs either. As consequence I will not apply logs or any other normalization technique.

I have droped categorical data so I will note use feature slips.

I tried to delete records with ourliers from the data set but the outcome was not good. Defined as below 0.1 quantile and above 0.9 quantile.

For varibale reduction I explored at the correlation matrix and tried to delete some correlated varibales but did not end up beeing helpul.

## Generating the predictions with the grid search in XGBoost

## XGBoost

In [130]:
%%time

model = XGBRegressor()

grid_values = {
    'n_estimators' : [100,150],
    'learning_rate': [0.1], #so called `eta` value
    'max_depth': [3],
    'random_state' : [seed],
    'sampling_method': ['uniform', 'gradient_based']
}


grid_xgb_acc = GridSearchCV(model, param_grid = grid_values, cv=5, scoring='neg_mean_squared_error')

grid_xgb_acc.fit(X_train, y)

y_pred_acc = grid_xgb_acc.predict(X_test)

print("MSE::{}".format(grid_xgb_acc.best_score_))
print("Best Hyperparameters::\n{}".format(grid_xgb_acc.best_params_))

MSE::-23.919689131887147
Best Hyperparameters::
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 170, 'random_state': 54, 'sampling_method': 'uniform'}
CPU times: user 6min 46s, sys: 1.65 s, total: 6min 47s
Wall time: 1min 9s


In [131]:
pred_df = pd.DataFrame(data_test['icustay_id'].copy())
pred_df['LOS'] = y_pred_acc
pred_df.to_csv('XGB_Prediction_kaggle', index=False)